In [2]:
import time
import os
import math
import numpy as np
import cv2
import pandas as pd
import pyautogui as pa

from lib.auto_GUI.auto_GUI_base import AutoGUIBase
from lib.auto_GUI.auto_PhotoZ import AutoPhotoZ

In [5]:
# Load data, not from Drive for Desktop since PhotoZ complains about loading zda from Drive
data_dir = "C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/mm_hidden/"
csv_filename = 'velocity_data.csv'

enable_photoZ_interaction = True
initialize_photoZ = True

# load the all-data CSV dir
df = pd.read_csv(data_dir + csv_filename)
df = pd.DataFrame(df)

skip_recs = [] # ['02-01-05_L23_interlaminar_intralaminar', '03-01-06_L4_intralaminar']

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/mm_hidden/velocity_data.csv'

In [6]:
# set up PhotoZ (open it manually)
if enable_photoZ_interaction:
    aPhz = AutoPhotoZ(data_dir=data_dir)
    if initialize_photoZ:
        aPhz.prepare_photoZ()

In [4]:
laminar_type_counter = [0, 0, 0]  # intra, inter, intra-inter folder counts
window_dt_jiggle = [-1, 0, 1]
if not enable_photoZ_interaction:
    skip_recs = []
for subdir, dirs, files in os.walk(data_dir):
    if subdir in skip_recs:
        continue
    else:
        skip_recs.append(subdir)
    zda_file = None
    roi_files = []
    roi_files_spare = []
    
    # locate important file names
    for f in files:
        if f.endswith('.zda'):
            zda_file = f
        if f.startswith('rois_'):
            if f.endswith('laminar.dat'):
                roi_files.append(f)
            else:
                roi_files_spare.append(f)
        if zda_file is not None and len(roi_files) == 2:
            break
    if len(roi_files) < 1:
        roi_files = roi_files_spare
        if len(roi_files) > 0:
            roi_files = [roi_files[0]]
        
    if zda_file is not None:
        rec_id = zda_file.split('.')[0]
        
        # alias the rec_id
        if '_aka_' in subdir:
            rec_id = subdir.split('_aka_')[0][-8:].replace("-", "_")
            print("aliased:", rec_id)
            
        # count type of folder
        if 'intralaminar' in subdir and 'interlaminar' in subdir:
            laminar_type_counter[2] += 1
        elif 'intralaminar' in subdir:
            laminar_type_counter[0] += 1
        elif 'interlaminar' in subdir:
            laminar_type_counter[1] += 1
            
        aPhz = AutoPhotoZ(data_dir=subdir)
        
        # open the PhotoZ file
        if enable_photoZ_interaction:
            aPhz.select_PhotoZ()
            
            print("\n\nOpening", zda_file)
            aPhz.open_zda_file(subdir + "/" + zda_file)
        
        date = subdir.split("/")[-1].split('\\')[0].replace('-', '_')
        date = date[5:] + '_' + date[:4]
        print(subdir)
        for roi_file in roi_files:
            print(roi_file)
            
            # open roi file
            if enable_photoZ_interaction:
                aPhz.select_roi_tab()
                aPhz.open_roi_file(roi_file)
            
            # Now load the data from the CSV needed to know the monosynaptic window
            lam_type = None
            lam_type_subset = df[(df['Date'] == date) 
                               & (df['Slice_Loc_Run'] == rec_id)]
            lam_type = list(lam_type_subset.Laminar.unique())

            if 'interlaminar' in roi_file and any(["Inter" in x for x in lam_type]):
                lam_type = ['Inter', 'Inter_L23', 'Inter_L5']
            elif 'intralaminar' in roi_file and "Intra" in lam_type:
                lam_type = ['Intra']
            elif 'interlaminar' in subdir and any(["Inter" in x for x in lam_type]):
                lam_type = ['Inter', 'Inter_L23', 'Inter_L5']
            elif 'intralaminar' in subdir and "Intra" in lam_type:
                lam_type = ['Intra']
            else:
                print("For " + " ".join(lam_type) + " for roi_file " + roi_file + " \n\tsubdir " + subdir + " rec " + rec_id)
                raise Exception("What is the laminar type? Unknown")
            mono_window_full = df[(df['Date'] == date) 
                             & (df['Laminar'].isin(lam_type))
                             & (df['Slice_Loc_Run'] == rec_id)]
            mono_window = pd.DataFrame(mono_window_full,
                                       columns = ['Date', 'Laminar',
                                                'Mono_Resp_Start_Time',
                                                'Mono_Resp_End_Time'])
            if mono_window.size < 1:
                raise Exception("(No row) Check laminar value " + " ".join(lam_type) + " for date " + date + " rec " + rec_id)
                continue
            mono_window = mono_window.drop_duplicates()
            if mono_window.shape[0] != 1:
                print(mono_window, mono_window.shape)
                raise Exception("Check laminar value " + " ".join(lam_type) + " for date " + date + " rec " + rec_id)
            
            # calculate and set measure window
            start = int(mono_window['Mono_Resp_Start_Time'] * 2)
            width = int(mono_window['Mono_Resp_End_Time'] * 2 - start)
            
            for jiggle_dt in window_dt_jiggle:
                jig_width = width + jiggle_dt
                filename_end = lam_type[0] + '.dat'
                if jiggle_dt != 0:
                    filename_end = lam_type[0] + "jig" + str(jiggle_dt) + '.dat'
                    
                if enable_photoZ_interaction:
                    aPhz.set_measure_window(start, jig_width)
                else:
                    print("Measure window:", start, jig_width)

                # Save values to file via PhotoZ: monosynaptic latency, max amp, SNR, peak time
                latency_filename = subdir + "/" + "mono_latency_" + filename_end
                print("Latency file dst:", latency_filename)
                if enable_photoZ_interaction:
                    aPhz.select_latency_trace_value()
                    aPhz.save_trace_values(latency_filename)

                maxamp_filename = subdir + "/" + "mono_maxamp_" + filename_end
                print("maxamp_filename dst:", maxamp_filename)
                if enable_photoZ_interaction:
                    aPhz.select_maxamp_trace_value()
                    aPhz.save_trace_values(maxamp_filename)

                snr_filename = subdir + "/" + "mono_snr_" + filename_end
                print("snr_filename dst:", snr_filename)
                if enable_photoZ_interaction:
                    aPhz.select_SNR_trace_value()
                    aPhz.save_trace_values(snr_filename)

                peaktime_filename = subdir + "/" + "mono_peaktime_" + filename_end
                print("peaktime_filename dst:", peaktime_filename)
                if enable_photoZ_interaction:
                    aPhz.select_peaktime_trace_value()
                    aPhz.save_trace_values(peaktime_filename)

                intermed_files = [latency_filename, maxamp_filename, snr_filename, peaktime_filename]
                col_names = ['Latency_Mono_Raw', 'Amp_Mono', 'SNR_Mono', 'Peak_Time_Mono']
                if jiggle_dt != 0:
                    col_names = [col + "_jig" + str(jiggle_dt) for col in col_names]

                #print("BEFORE WRITING")
                #print(df[(df['Date'] == date) 
                #       & (df['Laminar'].isin(lam_type))
                #       & (df['Slice_Loc_Run'] == rec_id)][col_names])

                # open these various files and store them to the pandas dataframe
                for i in range(len(intermed_files)):
                    intermed_file = intermed_files[i]
                    col_name = col_names[i]
                    intermed_df = pd.read_csv(intermed_file,
                                             sep='\t',
                                             header=None,
                                             names=['Index',  'Values'])

                    row_indices = df.index[(df['Date'] == date) 
                     & (df['Laminar'].isin(lam_type))
                     & (df['Slice_Loc_Run'] == rec_id)].tolist()

                    if intermed_df['Values'].shape[0] == len(row_indices):
                        for j in range(len(row_indices)):
                            row_idx = row_indices[j]
                            df.loc[row_idx,[col_name]] = intermed_df['Values'][j]
                    else:
                        print("Could not write values for " + " ".join(lam_type) 
                              + " for date " + date + " rec " + rec_id
                              + "col name" + col_name)
                        print(df[(df['Date'] == date) 
                         & (df['Laminar'].isin(lam_type))
                         & (df['Slice_Loc_Run'] == rec_id)])
                        print(intermed_df['Values'])

                # print("\n\nNum empty cells ", df[['Latency_Mono_Raw', 'Amp_Mono', 'SNR_Mono', 'Peak_Time_Mono']].count())
                #print("AFTER WRITE")
                #print(df[(df['Date'] == date) 
                #       & (df['Laminar'].isin(lam_type))
                #       & (df['Slice_Loc_Run'] == rec_id)][col_names])


C:/Users/jjudge3/Desktop/Data/Kate's/02-02-23\converted_zda
rois_01_to_13.dat
For  for roi_file rois_01_to_13.dat 
	subdir C:/Users/jjudge3/Desktop/Data/Kate's/02-02-23\converted_zda rec 01_01_23


Exception: What is the laminar type? Unknown

In [ ]:
# counts of inter, intra, inter-intra measurements
print(laminar_type_counter, "(#intra, #inter, #inter-intra)")
print("P(intra|inter) =", 
      round(laminar_type_counter[2] / (laminar_type_counter[0] + laminar_type_counter[2]),2) )
print("P(inter|intra) =", 
      round(laminar_type_counter[2] / (laminar_type_counter[1] + laminar_type_counter[2]),2) )

In [ ]:
# write the pandas dataframe back to csv
csv_filename = 'raw_velocity_data_JJ_edited.csv'
df.to_csv(data_dir + csv_filename, index=False)

In [ ]:
# Compare jiggled windows
col_names = ['Latency_Mono_Raw', 'Amp_Mono', 'SNR_Mono', 'Peak_Time_Mono']
tolerance = 0.05

n_viol = [0, 0, 0, 0]
n_total = 0

for j in range(len(col_names)):
    col = col_names[j]
    for idx in df.index:
        
        jiggled_values = []
        for jiggle_dt in window_dt_jiggle:
            if jiggle_dt != 0:
                v = df[col + "_jig" + str(jiggle_dt)][idx]
            else:
                v = df[col][idx]
            jiggled_values.append(v)
            
        for i in range(len(jiggled_values)-1):
            v_next = jiggled_values[i+1]
            v = jiggled_values[i]
            n_total += 1
            if np.abs((v_next - v) / v) > tolerance:
                print("TOLERANCE VIOLATION from i =", i, "to next; ", col, v_next, v)
                n_viol[j] += 1
    
print("% consistent to tolerance:", (n_total - sum(n_viol)) / n_total )
print(col_names)
print(n_viol)

In [6]:
# Compare KS and JJ csv files
ks_csv_filename = 'raw_velocity_data_KS.csv'
ks_df = pd.read_csv(data_dir + ks_csv_filename)
ks_df = pd.DataFrame(ks_df)
col_names = ['Latency_Mono_Raw', 'Amp_Mono', 'SNR_Mono', 'Peak_Time_Mono']
tolerance = 0.05

n_viol = [0, 0, 0, 0]
n_total = 0

for idx in ks_df.index:
    for i in range(len(col_names)):
        col = col_names[i]
        n_total += 1
        v = df[col][idx]
        ks_v = ks_df[col][idx]
        if v == 0:
            if ks_df[col][idx] == 0:
                continue
            v = 0.000000001
        if col == 'Amp_Mono':
            v /= 1000
        if col == 'Latency_Mono_Raw':
            v -= 46.5
            ks_v -= 46.5
            
        # print(col, v, ks_df[col][idx])
        if np.abs((ks_v - v) / v) > tolerance:
            print("TOLERANCE VIOLATION", col, ks_v, v)
            n_viol[i] += 1

print("% consistent to tolerance:", (n_total - sum(n_viol)) / n_total )
print(col_names)
print(n_viol)

% consistent to tolerance: 1.0
['Latency_Mono_Raw', 'Amp_Mono', 'SNR_Mono', 'Peak_Time_Mono']
[0, 0, 0, 0]


In [14]:
# Compare Synapticity classifications
n_total = 0
n_viol = 0
csv_filename = 'raw_velocity_data_JJ_KS_compare.csv'
df_edited = pd.read_csv(data_dir + csv_filename)
df_edited = pd.DataFrame(df_edited)
cause_counts = [0,0,0]

for idx in ks_df.index:
    col = 'Synaptic'
    v = df_edited[col][idx]
    ks_v = ks_df[col][idx]
    if not type(ks_v) == str:
        ks_v = ''
    if not type(v) == str:
        v = ''
    if v == '' or ks_v == '':
        continue
    n_total += 1
    if v == 'None':
        v = 'No_resp'
    if v == 'Direct Stimulation':
        v = 'Direct_resp'
    
    if ks_v != v:
        print("Classes not matched", ks_v, v)
        if "Disyn" in ks_v and "Mono" in v:
            cause_counts[0] += 1
        elif "Dir" in v or "Dir" in ks_v:
            cause_counts[1] += 1
        elif "No_resp" in v or "No_resp" in ks_v:
            cause_counts[2] += 1
        n_viol += 1

print(n_viol, "violations out of", n_total, ",", round(n_viol/n_total, 2) )
print("cause_counts", [x/n_viol for x in cause_counts])

Classes not matched Monosynaptic Direct_resp
Classes not matched Monosynaptic Direct_resp
Classes not matched Monosynaptic Direct_resp
Classes not matched No_resp Disynaptic
Classes not matched Monosynaptic Direct_resp
Classes not matched Monosynaptic Direct_resp
Classes not matched Disynaptic Monosynaptic
Classes not matched Disynaptic Monosynaptic
Classes not matched Disynaptic Calc error
Classes not matched Disynaptic Calc error
Classes not matched No_resp Disynaptic
Classes not matched No_resp Disynaptic
Classes not matched Latency_calc_error Monosynaptic
Classes not matched Monosynaptic Latency_calc_error
Classes not matched No_resp Monosynaptic
Classes not matched No_resp Monosynaptic
Classes not matched No_resp Monosynaptic
Classes not matched No_resp Latency_calc_error
Classes not matched No_resp Latency_calc_error
Classes not matched No_resp Monosynaptic
Classes not matched No_resp Monosynaptic
Classes not matched Monosynaptic Direct_resp
Classes not matched Monosynaptic Direc